In [1]:
%pip install osmnx geopandas shapely rtree pyproj pandas tqdm

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 4.9 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ------ --------------------------------- 1.0/6.3 MB 5.3 MB/s eta 0:00:01
   ----------- ---------------------------- 1.8/6.3 MB 4.9 MB/s eta 0:00:01
   ------------------- -------------------- 3.1/6.3 MB 5.1 MB/s eta 0:00:01
   ------------------------ --------------- 3.9/6.3 MB 5.2 MB/s eta 0:00:01
   ------------------------------- -------- 5.0/6.3 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------  6.3/6.3 MB 5.1

In [3]:
import os
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from tqdm import tqdm

import osmnx as ox
# configure osmnx
# Speed & caching settings
ox.settings.use_cache = True
ox.settings.cache_folder = "./osm_cache"
ox.settings.log_console = False
ox.settings.timeout = 180
# ox.settings.overpass_endpoint = "https://overpass.kumi.systems/api/interpreter"
ox.settings.overpass_endpoint = "https://overpass.private.coffee/api/interpreter"

In [4]:
stations = [
    ("Anand Vihar, Delhi (DPCC / CPCB)", 28.6473, 77.3185, "CPCB list"),
    ("R K Puram, Delhi (DPCC)", 28.5643, 77.1818, "paper"),
    ("Punjabi Bagh, Delhi (DPCC)", 28.6678, 77.1213, "CPCB list"),
    ("Mandir Marg, Delhi (DPCC)", 28.6274, 77.2010, "GIS study"),
    ("Vikas Sadan Gurugram - HSPCB", 28.4601, 77.0318, "amazon link?"),
    ("Sector-51 Gurugram - HSPCB", 28.4287, 77.0706, "env clearance doc"),
    ("Sector 62, Noida", 28.6195, 77.3618, "UPPCB"),
    ("Sector 125, Noida", 28.5376, 77.3338, "UPPCB"),
]

df_stations = pd.DataFrame(stations, columns=["station", "lat", "lon", "notes"])
gdf_stations = gpd.GeoDataFrame(df_stations,
                                geometry=gpd.points_from_xy(df_stations.lon, df_stations.lat),
                                crs="EPSG:4326")


In [5]:
df_stations

,station,lat,lon,notes
0,"Anand Vihar, Delhi (DPCC / CPCB)",28.6473,77.3185,CPCB list
1,"R K Puram, Delhi (DPCC)",28.5643,77.1818,paper
2,"Punjabi Bagh, Delhi (DPCC)",28.6678,77.1213,CPCB list
3,"Mandir Marg, Delhi (DPCC)",28.6274,77.2010,GIS study
4,Vikas Sadan Gurugram - HSPCB,28.4601,77.0318,amazon link?
5,Sector-51 Gurugram - HSPCB,28.4287,77.0706,env clearance doc
6,"Sector 62, Noida",28.6195,77.3618,UPPCB
7,"Sector 125, Noida",28.5376,77.3338,UPPCB


In [6]:
gdf_stations

,station,lat,lon,notes,geometry
0,"Anand Vihar, Delhi (DPCC / CPCB)",28.6473,77.3185,CPCB list,POINT (77.3185 28.6473)
1,"R K Puram, Delhi (DPCC)",28.5643,77.1818,paper,POINT (77.1818 28.5643)
2,"Punjabi Bagh, Delhi (DPCC)",28.6678,77.1213,CPCB list,POINT (77.1213 28.6678)
3,"Mandir Marg, Delhi (DPCC)",28.6274,77.2010,GIS study,POINT (77.201 28.6274)
4,Vikas Sadan Gurugram - HSPCB,28.4601,77.0318,amazon link?,POINT (77.0318 28.4601)
5,Sector-51 Gurugram - HSPCB,28.4287,77.0706,env clearance doc,POINT (77.0706 28.4287)
6,"Sector 62, Noida",28.6195,77.3618,UPPCB,POINT (77.3618 28.6195)
7,"Sector 125, Noida",28.5376,77.3338,UPPCB,POINT (77.3338 28.5376)


In [7]:
%pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [47]:
# import shapely
# p = shapely.Point(77.2949, 28.6559)  # Anand Vihar (lon, lat)
# buf = make_buffer(p, meters=10)
# print(buf.bounds)

In [8]:
def get_utm_crs_for_point(lat, lon):
    """Return an appropriate UTM CRS for given lat/lon using osmnx utility."""
    try:
        # osmnx returns pyproj CRS
        return ox.projection.get_utm_crs(lat, lon)
    except Exception:
        # fallback: approximate by EPSG:3857 for metric operations (less accurate near poles, ok here)
        import pyproj
        return pyproj.CRS.from_epsg(3857)

# def make_buffer(point_geom, meters=500):
#     """
#     Create a buffer polygon (in WGS84 returned) around a point geometry with radius in meters.
#     Steps: project to appropriate UTM -> buffer -> reproject to WGS84.
#     """
#     lat, lon = point_geom.y, point_geom.x
#     utm_crs = get_utm_crs_for_point(lat, lon)
#     # project point to utm
#     g = gpd.GeoSeries([point_geom], crs="EPSG:4326").to_crs(utm_crs)
#     buf = g.buffer(meters).iloc[0]
#     buf_wgs84 = gpd.GeoSeries([buf], crs=utm_crs).to_crs("EPSG:4326").iloc[0]
#     return buf_wgs84

def make_buffer(point_geom, meters=500):
    """
    Create a small, valid buffer polygon (WGS84) around a point.
    Works robustly for OSMnx / Overpass.
    """
    lat, lon = point_geom.y, point_geom.x
    utm_crs = get_utm_crs_for_point(lat, lon)
    
    # project to local UTM, buffer, and reproject back
    point_gdf = gpd.GeoSeries([point_geom], crs="EPSG:4326").to_crs(utm_crs)
    buffer_geom = point_gdf.buffer(meters).to_crs("EPSG:4326").iloc[0]

    # Clean geometry and force validity
    buffer_geom = buffer_geom.buffer(0)
    if not buffer_geom.is_valid:
        buffer_geom = buffer_geom.buffer(0)
    
    # Clip insane coordinates just in case (fixes overpass max area issue)
    minx, miny, maxx, maxy = buffer_geom.bounds
    if (maxx - minx) > 0.05 or (maxy - miny) > 0.05:  # ~5 km threshold
        raise ValueError(f"Buffer for {point_geom} too large: {maxx-minx:.4f}, {maxy-miny:.4f}")

    return buffer_geom


def area_m2(geom):
    """Return approximate area in m^2 by projecting geometry to UTM chosen for centroid."""
    lat, lon = geom.centroid.y, geom.centroid.x
    utm_crs = get_utm_crs_for_point(lat, lon)
    g = gpd.GeoSeries([geom], crs="EPSG:4326").to_crs(utm_crs)
    return g.iloc[0].area

In [9]:
# ---------------------------
# 3) Road / traffic features via OSM
# ---------------------------
major_highways = ['motorway','trunk','primary','secondary','tertiary']

def compute_road_features(point_row, buffer_poly):
    """
    point_row: a Series with columns ['station','lat','lon','geometry']
    buffer_poly: shapely polygon in WGS84
    Returns:
      dict with distance_to_major_road (m), total_road_length_m, major_road_length_m
    """
    # bbox for Overpass query (use a small bbox around the buffer to limit query)
    minx, miny, maxx, maxy = buffer_poly.bounds
    bbox = (minx, miny, maxx, maxy)  # west, south, east, north
    tags = {'highway': True}
    print(f"Trying with BBOX: {bbox} and tags: {tags}")

    # Query OSM for highway features within bbox
    try:
        ways = ox.features_from_bbox(bbox, tags)
    except Exception as e:
        # print error and return NaNs / zeros
        print(f"OSM roads fetch failed for {point_row.station}: {e}")
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Keep only geometries that are linestring-like
    if ways.empty or 'geometry' not in ways.columns:
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    ways = ways[ways.geometry.notnull()].copy()
    # if 'highway' column missing, try other representations
    if 'highway' not in ways.columns:
        # sometimes the highway tag is in 'tags' or dtype object - attempt to extract
        ways['highway'] = ways.apply(lambda r: r.get('highway') if isinstance(r, dict) else None, axis=1)

    # explode multi-geometries
    ways = ways.explode(ignore_index=True)
    ways = ways[ways.geometry.type.isin(['LineString','MultiLineString'])]
    if ways.empty:
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Intersect each geometry with buffer polygon
    ways['segment'] = ways.geometry.intersection(buffer_poly)
    ways = ways[~ways['segment'].is_empty].copy()
    if ways.empty:
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Project to UTM to measure length
    utm = get_utm_crs_for_point(point_row.lat, point_row.lon)
    segs = gpd.GeoSeries(ways['segment'].values, crs="EPSG:4326").set_crs("EPSG:4326").to_crs(utm)
    segs = segs[segs.notnull()]
    lengths = segs.length
    total_len = lengths.sum()

    # identify major segments by highway tag
    # highway tags can be strings or lists; convert and match
    def is_major_tag(val):
        if val is None:
            return False
        s = str(val).lower()
        return any(h in s for h in major_highways)

    # align list lengths
    # note: ways['highway'] can be scalar per feature, repeating to same index as segs
    ways = ways.reset_index(drop=True).loc[segs.index if hasattr(segs, "index") else ways.index]
    ways = ways.assign(length_m=lengths.values)
    ways['is_major'] = ways['highway'].apply(is_major_tag)
    major_len = ways.loc[ways['is_major'], 'length_m'].sum() if 'length_m' in ways.columns else 0.0

    # compute distance from station to nearest major road (projected)
    major_segs = gpd.GeoSeries(ways.loc[ways['is_major'], 'segment'].values,
                               crs="EPSG:4326").to_crs(utm)
    if not major_segs.empty:
        station_point_proj = gpd.GeoSeries([point_row.geometry], crs="EPSG:4326").to_crs(utm).iloc[0]
        dists = major_segs.distance(station_point_proj)
        min_dist = float(dists.min())
    else:
        min_dist = np.nan

    return {'distance_to_major_road': float(min_dist) if np.isfinite(min_dist) else np.nan,
            'total_road_length_m': float(total_len),
            'major_road_length_m': float(major_len)}


In [10]:

# ---------------------------
# 4) Land use features via OSM (proxy for LULC)
# ---------------------------
def compute_landuse_features(buffer_poly):
    """
    buffer_poly: shapely polygon in WGS84
    Returns: pct_green, pct_industrial, pct_residential (fractions 0..1)
    NOTE: OSM landuse is a proxy; for authoritative LULC, replace with raster zonal stats.
    """
    minx, miny, maxx, maxy = buffer_poly.bounds
    bbox = (minx, miny, maxx, maxy)  # west, south, east, north
    tags = {'landuse': True, 'leisure': True, 'natural': True}
    print(f"Trying with BBOX: {bbox} and tags: {tags}")
    
    try:
        polys = ox.features_from_bbox(bbox, tags)
    except Exception as e:
        print("OSM landuse fetch failed:", e)
        return {'pct_green': np.nan, 'pct_industrial': np.nan, 'pct_residential': np.nan}

    if polys.empty or 'geometry' not in polys.columns:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    polys = polys[polys.geometry.notnull()].explode(ignore_index=True)
    polys = polys[polys.geometry.type.isin(['Polygon','MultiPolygon'])].copy()
    if polys.empty:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    # compute intersection with buffer
    polys['intersect'] = polys.geometry.intersection(buffer_poly)
    polys = polys[~polys['intersect'].is_empty].copy()
    if polys.empty:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    # project to UTM for area calc
    centroid = buffer_poly.centroid
    utm = get_utm_crs_for_point(centroid.y, centroid.x)
    intersect_gs = gpd.GeoSeries(polys['intersect'].values, crs="EPSG:4326").to_crs(utm)
    areas = intersect_gs.area
    polys = polys.reset_index(drop=True).assign(area_m2=areas.values)
    total_area = polys['area_m2'].sum() if len(polys)>0 else 0.0
    if total_area <= 0:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    def area_by_keywords(keywords):
        mask = polys.apply(lambda r: any((kw in str(r.get('landuse','')).lower()) for kw in keywords) or
                                       any((kw in str(r.get('leisure','')).lower()) for kw in keywords) or
                                       any((kw in str(r.get('natural','')).lower()) for kw in keywords),
                           axis=1)
        return polys.loc[mask, 'area_m2'].sum()

    green_area = area_by_keywords(['forest','grass','park','recreation','garden','meadow','wood','green'])
    industrial_area = area_by_keywords(['industrial','quarry','landfill'])
    residential_area = area_by_keywords(['residential','housing','residential;apartments'])  # some tag variants

    return {'pct_green': float(green_area/total_area),
            'pct_industrial': float(industrial_area/total_area),
            'pct_residential': float(residential_area/total_area)}


In [11]:
# ---------------------------
# 5) Building morphology via OSM footprints
# ---------------------------
def compute_building_features(buffer_poly):
    """
    Returns building_density (fraction of buffer area that is building footprint),
    avg_building_area_m2, median_building_area_m2, building_count
    """
    minx, miny, maxx, maxy = buffer_poly.bounds
    bbox = (minx, miny, maxx, maxy)  # west, south, east, north
    tags = {'building': True}
    print(f"Trying with BBOX: {bbox} and tags: {tags}")

    try:
        blds = ox.features_from_bbox(bbox, tags)
    except Exception as e:
        print("OSM buildings fetch failed:", e)
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    if blds.empty or 'geometry' not in blds.columns:
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    blds = blds[blds.geometry.notnull()].explode(ignore_index=True)
    blds = blds[blds.geometry.type.isin(['Polygon','MultiPolygon'])].copy()
    if blds.empty:
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    blds['intersect'] = blds.geometry.intersection(buffer_poly)
    blds = blds[~blds['intersect'].is_empty].copy()
    if blds.empty:
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    centroid = buffer_poly.centroid
    utm = get_utm_crs_for_point(centroid.y, centroid.x)
    areas = gpd.GeoSeries(blds['intersect'].values, crs="EPSG:4326").to_crs(utm).area
    blds = blds.reset_index(drop=True).assign(area_m2=areas.values)
    total_bld_area = blds['area_m2'].sum()
    buf_area = area_m2(buffer_poly)
    building_density = float(total_bld_area / buf_area) if buf_area>0 else 0.0

    return {'building_density': building_density,
            'avg_building_area_m2': float(blds['area_m2'].mean()),
            'median_building_area_m2': float(blds['area_m2'].median()),
            'building_count': int(len(blds))}

In [15]:

# ---------------------------
# 6) Pipeline: run for each station
# ---------------------------
results = []
buffer_radius_m = 1000  # change to desired radius (meters)

print("Starting hyperlocal extraction for", len(gdf_stations), "stations. Buffer radius:", buffer_radius_m, "m")
for idx, row in tqdm(gdf_stations.reset_index(drop=True).iterrows(), total=len(gdf_stations)):
    station_name = row['station']
    pt = row.geometry
    try:
        buffer_poly = make_buffer(pt, meters=buffer_radius_m)
    except Exception as e:
        print(f"Failed to build buffer for {station_name}: {e}")
        buffer_poly = None

    if buffer_poly is None or buffer_poly.is_empty:
        # append empty features
        out = dict(row.drop(labels=['geometry']).to_dict())
        out.update({
            'distance_to_major_road': np.nan,
            'total_road_length_m': 0.0,
            'major_road_length_m': 0.0,
            'pct_green': np.nan,
            'pct_industrial': np.nan,
            'pct_residential': np.nan,
            'building_density': 0.0,
            'avg_building_area_m2': np.nan,
            'median_building_area_m2': np.nan,
            'building_count': 0
        })
        results.append(out)
        continue

    # Roads
    try:
        road_feats = compute_road_features(row, buffer_poly)
    except Exception as e:
        print(f"Error computing road features for {station_name}: {e}")
        road_feats = {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Landuse
    try:
        land_feats = compute_landuse_features(buffer_poly)
    except Exception as e:
        print(f"Error computing landuse for {station_name}: {e}")
        land_feats = {'pct_green': np.nan, 'pct_industrial': np.nan, 'pct_residential': np.nan}

    # Buildings
    try:
        bld_feats = compute_building_features(buffer_poly)
    except Exception as e:
        print(f"Error computing buildings for {station_name}: {e}")
        bld_feats = {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    out = dict(row.drop(labels=['geometry']).to_dict())
    out.update(road_feats)
    out.update(land_feats)
    out.update(bld_feats)
    results.append(out)

    # polite pause to avoid hammering Overpass (optional)
    time.sleep(1.0)

Starting hyperlocal extraction for 8 stations. Buffer radius: 1000 m


  0%|          | 0/8 [00:00<?, ?it/s]

Trying with BBOX: (77.30951684715879, 28.639416201119296, 77.32748315284118, 28.655183206312497) and tags: {'highway': True}
Trying with BBOX: (77.30951684715879, 28.639416201119296, 77.32748315284118, 28.655183206312497) and tags: {'landuse': True, 'leisure': True, 'natural': True}
Trying with BBOX: (77.30951684715879, 28.639416201119296, 77.32748315284118, 28.655183206312497) and tags: {'building': True}


 12%|█▎        | 1/8 [00:09<01:09,  9.93s/it]

Trying with BBOX: (77.1728168471588, 28.556409971200875, 77.19078315284119, 28.57218943733581) and tags: {'highway': True}
Trying with BBOX: (77.1728168471588, 28.556409971200875, 77.19078315284119, 28.57218943733581) and tags: {'landuse': True, 'leisure': True, 'natural': True}
Trying with BBOX: (77.1728168471588, 28.556409971200875, 77.19078315284119, 28.57218943733581) and tags: {'building': True}


...

 75%|███████▌  | 6/8 [06:15<02:10, 65.00s/it]

Trying with BBOX: (77.35281684715879, 28.6116141126284, 77.37078315284118, 28.627385295172896) and tags: {'highway': True}
Trying with BBOX: (77.35281684715879, 28.6116141126284, 77.37078315284118, 28.627385295172896) and tags: {'landuse': True, 'leisure': True, 'natural': True}
Trying with BBOX: (77.35281684715879, 28.6116141126284, 77.37078315284118, 28.627385295172896) and tags: {'building': True}


 88%|████████▊ | 7/8 [07:55<01:16, 76.26s/it]

Trying with BBOX: (77.3248168471588, 28.529707970638274, 77.34278315284119, 28.545491438254885) and tags: {'highway': True}
Trying with BBOX: (77.3248168471588, 28.529707970638274, 77.34278315284119, 28.545491438254885) and tags: {'landuse': True, 'leisure': True, 'natural': True}
Trying with BBOX: (77.3248168471588, 28.529707970638274, 77.34278315284119, 28.545491438254885) and tags: {'building': True}


100%|██████████| 8/8 [08:03<00:00, 60.48s/it]


In [16]:

# Build results DataFrame & some derived features
df_results = pd.DataFrame(results)
# derive fraction of major roads
df_results['major_road_fraction'] = df_results.apply(
    lambda r: float(r.major_road_length_m / r.total_road_length_m) if (r.total_road_length_m and not np.isnan(r.total_road_length_m)) else np.nan,
    axis=1
)

# Save CSV
# out_dir = '/mnt/data'
# os.makedirs(out_dir, exist_ok=True)
# out_csv = os.path.join(out_dir, 'delhi_hyperlocal_context_features.csv')
df_results.to_csv(f'delhi_hyperlocal_context_features_{buffer_radius_m}m.csv', index=False)
print(f"Saved results to: delhi_hyperlocal_context_features_{buffer_radius_m}m.csv")

Saved results to: delhi_hyperlocal_context_features_1000m.csv


In [14]:
df_results

,station,lat,lon,notes,distance_to_major_road,total_road_length_m,major_road_length_m,pct_green,pct_industrial,pct_residential,building_density,avg_building_area_m2,median_building_area_m2,building_count,major_road_fraction
0,"Anand Vihar, Delhi (DPCC / CPCB)",28.6473,77.3185,CPCB list,36.472419,16403.307118,5226.357820,0.044158,0.461872,0.173997,0.118089,1747.135092,597.210473,53,0.318616
1,"R K Puram, Delhi (DPCC)",28.5643,77.1818,paper,125.997238,23375.007044,5068.167200,0.019514,0.000000,0.638365,0.169456,734.123494,474.477886,181,0.216820
2,"Punjabi Bagh, Delhi (DPCC)",28.6678,77.1213,CPCB list,123.872699,16205.321772,1946.623106,0.182712,0.000000,0.817288,0.142625,306.403937,232.802704,365,0.120122
3,"Mandir Marg, Delhi (DPCC)",28.6274,77.2010,GIS study,293.086190,22356.630390,4631.179981,0.028210,0.000000,0.930702,0.196425,777.898137,486.630582,198,0.207150
4,Vikas Sadan Gurugram - HSPCB,28.4601,77.0318,amazon link?,31.276029,20632.212200,6338.127898,0.000000,0.000000,0.496910,0.232092,264.138976,146.429395,689,0.307196
5,Sector-51 Gurugram - HSPCB,28.4287,77.0706,env clearance doc,213.792151,18345.106427,2345.876114,0.005342,0.000000,0.819356,0.156403,882.314167,668.899360,139,0.127875
6,"Sector 62, Noida",28.6195,77.3618,UPPCB,183.931500,12321.370052,2984.402997,0.273016,0.000000,0.033268,0.127668,2944.379043,1775.219799,34,0.242214
7,"Sector 125, Noida",28.5376,77.3338,UPPCB,137.054350,9111.665622,4443.734134,0.337220,0.000000,0.000531,0.020742,650.585589,366.440154,25,0.487697
